In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import os
import numpy as np
import cv2
import random
import pickle
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical

random.seed(5)

In [2]:
tf.__version__

'2.1.0'

In [3]:
DATADIR = "./data/RAW/alphabet-dataset"
CATEGORIES = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "Y", "V", "W", "X", "Y", "Z"]
IMG_SIZE = 28


train_data = []

def create_train_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for file in os.listdir(path):
            file_array = cv2.imread(os.path.join(path, file), cv2.IMREAD_GRAYSCALE)
            file_array = cv2.resize(file_array, (IMG_SIZE, IMG_SIZE))
            train_data.append([file_array, class_num])

create_train_data()

In [4]:
random.shuffle(train_data)

In [5]:
X = []
y = []

In [6]:
for feature, label in train_data:
    X.append(feature)
    y.append(label)

In [7]:
print(len(X))
print(len(y))
print(len(CATEGORIES))

354302
354302
26


In [8]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [9]:
pickle_out = open("./data/CURATED/X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("./data/CURATED/y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()


# Preparation et entrainement du modele

In [ ]:
CATEGORIES = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "Y", "V", "W", "X", "Y", "Z"]

pickle_in = open("./data/CURATED/X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("./data/CURATED/y.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [10]:
X = X /255.0

y = to_categorical(y)

In [12]:
print(y[0:10])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [15]:
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(len(CATEGORIES)))
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(X, y, batch_size=64, epochs=10, validation_split=0.25)

Train on 265726 samples, validate on 88576 samples
Epoch 1/10
265726/265726 [==============================] - 28s 104us/sample - loss: 0.1323 - accuracy: 0.9628 - val_loss: 0.0674 - val_accuracy: 0.9817
Epoch 2/10
265726/265726 [==============================] - 26s 96us/sample - loss: 0.0539 - accuracy: 0.9849 - val_loss: 0.0504 - val_accuracy: 0.9868
Epoch 3/10
265726/265726 [==============================] - 26s 96us/sample - loss: 0.0393 - accuracy: 0.9887 - val_loss: 0.0491 - val_accuracy: 0.9866
Epoch 4/10
265726/265726 [==============================] - 26s 98us/sample - loss: 0.0302 - accuracy: 0.9910 - val_loss: 0.0388 - val_accuracy: 0.9897
Epoch 5/10
265726/265726 [==============================] - 26s 96us/sample - loss: 0.0240 - accuracy: 0.9925 - val_loss: 0.0400 - val_accuracy: 0.9905
Epoch 6/10
265726/265726 [==============================] - 28s 107us/sample - loss: 0.0199 - accuracy: 0.9937 - val_loss: 0.0405 - val_accuracy: 0.9900
Epoch 7/10
265726/265726 [=========

In [16]:
model.save('./data/OUTPUT/cnn_model.h5')

# Utilisation du modele

In [ ]:
model = load_model('./data/OUTPUT/cnn_model.h5')

In [14]:
CATEGORIES = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "Y", "V", "W", "X", "Y", "Z"]
IMG_SIZE = 28


def load(filename):
    file_array = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    file_array = cv2.resize(file_array, (IMG_SIZE, IMG_SIZE))
    np_image = np.array(file_array).reshape(-1, IMG_SIZE, IMG_SIZE, 1)/255.0
    return np_image

img = load('./data/RAW/alphabet-dataset/O/O-46.png')
classes = model.predict_classes(img)

print(CATEGORIES[classes[0]])

O
